In [ ]:
# |default_exp agent.dqn

In [ ]:
# |export
import logging
from copy import deepcopy
from functools import cached_property, partial

import numpy as np
import torch
from fastai.optimizer import Adam
from fastcore.basics import class2attr, store_attr

# DQN Agent
> Agent allowing to pick an action and fit one batch for training using DQN algorithm

In [ ]:
# |export
class Exploration:
    def __init__(self):
        self.action_tier = 1.0

    @property
    def name(self):
        return class2attr(self, "Exploration")

    def __call__(self, action_values):
        n_action = len(action_values)
        argsort = action_values.argsort()
        argmax = argsort[-1].item()
        rank = argsort.argsort().double()
        action = self.select_action(action_values, n_action, argmax, rank)
        self.action_tier = (rank[action] / (len(rank) - 1)).item()
        return action

    def select_action(self, action_values, n_action, argmax, rank):
        raise NotImplementedError()

    def state_dict(self):
        return dict(exploration=self.name, action_tier=self.action_tier)


class EpsilonGreedyExploration(Exploration):
    def __init__(self, epsilon=0.1):
        super().__init__()
        store_attr()

    def select_action(self, action_values, n_action, argmax, rank):
        if np.random.random() < self.epsilon:
            return np.random.choice(n_action)
        return argmax


class ResmaxExploration(Exploration):
    def __init__(self, eta):
        super().__init__()
        store_attr()
        self.eta_inv = eta**-1

    def select_action(self, action_values, n_action, argmax, rank):
        p = 1 / (n_action + self.eta_inv * (action_values[argmax] - action_values))
        p = p.double()
        p[argmax] += 1 - p.sum()
        return np.random.choice(n_action, p=p)


class RankSoftmaxExploration(Exploration):
    def __init__(self, tau=1.0):
        super().__init__()
        store_attr()

    def select_action(self, action_values, n_action, argmax, rank):
        p = (rank**self.tau).softmax(0)
        return np.random.choice(n_action, p=p)


class NoExploration(Exploration):
    def select_action(self, action_values, n_action, argmax, rank):
        return argmax

In [ ]:
# |export
class DqnAgent:
    tags = ["DQN"]
    """ Require use of DiscreteActionHandler in Tmenv """

    def __init__(
        self,
        model,
        exploration=EpsilonGreedyExploration(),
        device="cpu",
        gamma=0.99,
        update_target_every=320,
        optimizer_fn=partial(Adam, lr=1e-5, wd=1e-5),
        loss_fn=torch.nn.SmoothL1Loss(reduction="none"),
    ):
        store_attr()
        self.net_online = model.to(device=device)
        self.net_online.share_memory()

    @torch.no_grad()
    def select_action(self, state, evaluation=False):
        self.net_online.eval()
        action_values = self.net_online(state).squeeze().cpu()
        if evaluation:
            return action_values.argmax()
        return self.exploration(action_values)

    @cached_property
    def net_target(self):
        net_target = deepcopy(self.net_online).eval()
        for p in net_target.parameters():
            p.requires_grad = False
        return net_target

    @cached_property
    def nb_batch(self):
        return 0

    @cached_property
    def optimizer(self):
        return self.optimizer_fn(self.net_online.parameters())

    @cached_property
    def scaler(self):
        return torch.cuda.amp.GradScaler()

    def _td_estimate(self, state, action):
        return self.net_online(state)[np.arange(0, len(action)), action]

    @torch.no_grad()
    def _td_target(self, reward, next_state, done):
        next_state_Q = self.net_online(next_state)
        best_action = torch.argmax(next_state_Q, axis=1)
        next_Q = self.net_target(next_state)[
            np.arange(0, len(best_action)), best_action
        ]
        return (
            torch.tensor(reward, device=self.device)
            + (1 - torch.tensor(done, device=self.device, dtype=torch.float))
            * self.gamma
            * next_Q
        ).float()

    def _sync_Q_target(self):
        self.net_target.load_state_dict(self.net_online.state_dict())

    def set_lr(self, lr):
        self.optimizer.set_hyper("lr", lr)

    def fit_one_batch(self, batch, weights):
        state, next_state, action, reward, done = zip(*batch)

        self.net_online.train()
        with torch.cuda.amp.autocast():
            td_est = self._td_estimate(state, action)
            td_tgt = self._td_target(reward, next_state, done)
            loss = (self.loss_fn(td_est, td_tgt) * weights).mean()

        self.optimizer.zero_grad()
        self.scaler.scale(loss).backward()
        self.scaler.step(self.optimizer)
        self.scaler.update()

        self.nb_batch += 1
        if self.nb_batch % self.update_target_every == 0:
            self._sync_Q_target()

        errors = torch.abs(td_est - td_tgt).detach()
        metrics = dict(
            mean_errors=errors.mean().item(),
            loss=loss.item(),
            lr=self.optimizer.param_groups[0]["lr"],
        )

        return errors, metrics

    def state_dict(self):
        return dict(
            net_online=self.net_online.state_dict(),
        )

    def load_state_dict(self, state_dict):
        self.net_online.load_state_dict(state_dict["net_online"], strict=False)
        self.net_online.to(self.device)

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()